In [2]:
import os
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")
from plotly.subplots import make_subplots
import plotly.graph_objects as go

plt.rcParams["figure.figsize"] = (20, 9)


### Aggregate all the csv files

##### Read all files from folder and sort them as per the file name
##### Merge all the files

In [12]:
# csvfiles = ['Day1_driver1.csv','Day1_driver2.csv','Day2_driver2_1.csv','Day2_driver1_1.csv','Day2_driver2_2.csv','Day2_driver1_2.csv','Day3_driver2_1.csv','Day3_driver1_1.csv','Day3_driver2_2.csv','Day3_driver1_2.csv','Day3_driver1_3.csv']
# csvfiles = ['Day1_final_data.csv']
path = f"./Data_collection/19_Nov/"
csvfiles = []
for x in os.listdir(path):
    if x.endswith(".csv"):
            csvfiles.append(x)
csvfiles.sort(key = lambda f: int(''.join(filter(str.isdigit, f))))
blink_avg_duration_per_minute = []
perclose_per_minute = []
missing_frames_per_minutes = []
number_of_blinks = []
avg_EAR = []
blink_duration = pd.DataFrame(columns=["Start_blink", "End_blink", "Blink_frame", "No_of_frames"])
perclos_value = list()
for x in csvfiles:
    df_alert = pd.read_csv(path+x)
    missing_frames_per_minutes.append(df_alert[df_alert["NewEAR"] == 0].shape[0])
    df_alert = df_alert.loc[(df_alert["EAR"] != 0)]
    df_alert = df_alert.reset_index()  # reindexing to identify the blink duration
    df_alert["NewEMA"] = df_alert["NewEAR"].ewm(4).mean()  # exponetial weighted average moving a
    startframe = list()
    endframe = list()
    frame_no = []
    # print(x)
    if(df_alert.shape[0]>4):
        for i in range(df_alert.shape[0]):
            if df_alert["NewEAR"].iloc[i] < df_alert["NewEAR"].iloc[i-4]*0.75 and i > 4 and (df_alert["FN"].iloc[i] - df_alert["FN"].iloc[i-1]) < 2 and df_alert["NewEAR"].iloc[i]<0.4:
                if len(frame_no) == 0 or (df_alert["FN"].iloc[i] - frame_no[-1] > 8):
                    frame_no.append(df_alert["FN"].iloc[i])
                else:
                    if (df_alert["NewEAR"].iloc[i] < df_alert["NewEAR"].iloc[i-1]):
                        frame_no[-1] = df_alert["FN"].iloc[i]
    number_of_blinks.append(len(frame_no))
    # print(frame_no)
    for i in range(len(frame_no)):
        start_frame, end_frame, blinked_frame = 0, 0, 0
        blinked_frame = frame_no[i]
        earemv = df_alert["NewEAR"].loc[df_alert["FN"]== blinked_frame].iloc[0]
        # print(i)
        if i == 0:
            sf = 0
        else:
            # print("end_blink",blink_duration["End_blink"].iloc[i-1])
            sf = df_alert.index[df_alert["FN"] == blink_duration["End_blink"].iloc[i-1]][0]
        # 
        if i == len(frame_no)-1:
            en = df_alert.shape[0]
        else:
            en = df_alert.index[df_alert["FN"] == frame_no[i+1]][0]
        # print(sf)
        # print("start {} and end frame {}".format(sf,en))
        
        for j in range(df_alert.index[df_alert["FN"] == frame_no[i]][0], sf, -1):
        # print(j)
            if (df_alert["NewEAR"].iloc[j] > (earemv/0.75)) or df_alert["NewEAR"].iloc[j] > 0.40:
                start_frame = df_alert["FN"].iloc[j]
                break
            else:
                start_frame = df_alert["FN"].iloc[j]
            # print("start frame",start_frame)
                continue
        for j in range(df_alert.index[df_alert["FN"] == frame_no[i]][0], en):
        # print(df_alert["NewEMA"].iloc[j])
            if ((df_alert["NewEAR"].iloc[j]) > (earemv/0.75)) or (df_alert["NewEAR"].iloc[j] > 0.40):
                end_frame = df_alert["FN"].iloc[j]
                break
            else:
                end_frame = df_alert["FN"].iloc[j]
                continue
            
        if end_frame-start_frame > 100:
            duration = 100
        else:
            duration = end_frame-start_frame
        blink_duration.loc[blink_duration.shape[0]] = [start_frame, end_frame, blinked_frame, duration]
    
    if len(blink_duration) == 0:
        blink_avg_duration_per_minute.append(0)
    else:
        blink_avg_duration_per_minute.append(sum(blink_duration["No_of_frames"])/blink_duration.shape[0])
    no_of_blinks = blink_duration.shape[0]
    perclos_value.append(sum(blink_duration["No_of_frames"])/1800)
    avg_EAR.append(sum(df_alert["NewEAR"])/1800)    
    # file = blink_duration.to_csv(x.split(".")[0]+"_Blink_Duration.csv")
    frame_no.clear()
    blink_duration = blink_duration.drop(blink_duration.index[0:])

Data_day1_analysis = pd.DataFrame(columns=["File_name","MissingFrames_per_min","Blink_per_min", "blink_duration_per_min", "Perclose_per_min", "Avg_EAR"])
Data_day1_analysis["File_name"] = csvfiles
Data_day1_analysis["MissingFrames_per_min"] = missing_frames_per_minutes
Data_day1_analysis["Blink_per_min"] = number_of_blinks
Data_day1_analysis["blink_duration_per_min"] = blink_avg_duration_per_minute
Data_day1_analysis["Perclose_per_min"] = perclos_value
Data_day1_analysis["Avg_EAR"] = avg_EAR
Data_day1_analysis.to_csv("Day2_data_analysis_new.csv")
print(len(number_of_blinks))
print(len(blink_avg_duration_per_minute))
print(len(perclos_value))
print(len(missing_frames_per_minutes))
# file = df_alert.to_csv("Test.csv")


466
466
466
466


In [6]:
pd.set_option("display.max_rows", None)

#### Based on EMA

In [7]:
# csvfiles = ['Day1_driver1.csv','Day1_driver2.csv','Day2_driver2_1.csv','Day2_driver1_1.csv','Day2_driver2_2.csv','Day2_driver1_2.csv','Day3_driver2_1.csv','Day3_driver1_1.csv','Day3_driver2_2.csv','Day3_driver1_2.csv','Day3_driver1_3.csv']
# csvfiles = ['Day1_final_data.csv']
path = f"./Data_collection/23_Nov/"
csvfiles = []
for x in os.listdir(path):
    if x.endswith(".csv"):
            csvfiles.append(x)
csvfiles.sort(key = lambda f: int(''.join(filter(str.isdigit, f))))
blink_avg_duration_per_minute = []
perclose_per_minute = []
missing_frames_per_minutes = []
number_of_blinks = []
blink_duration = pd.DataFrame(columns=["Start_blink", "End_blink", "Blink_frame", "No_of_frames"])
perclos_value = list()
for x in csvfiles:
    df_alert = pd.read_csv(path+x)
    missing_frames_per_minutes.append(df_alert[df_alert["NewEAR"] == 0].shape[0])
    df_alert = df_alert.loc[(df_alert["EAR"] != 0)]
    df_alert = df_alert.reset_index()  # reindexing to identify the blink duration
    df_alert["NewEMA"] = df_alert["NewEAR"].ewm(4).mean()  # exponetial weighted average moving a
    startframe = list()
    endframe = list()
    frame_no = []
    # print(x)
    if(df_alert.shape[0]>4):
        for i in range(df_alert.shape[0]):
            if df_alert["NewEMA"].iloc[i] < df_alert["NewEMA"].iloc[i-4]*0.84 and i > 4 and (df_alert["FN"].iloc[i] - df_alert["FN"].iloc[i-1]) < 2 and df_alert["NewEAR"].iloc[i]<0.4:
                if len(frame_no) == 0 or (df_alert["FN"].iloc[i] - frame_no[-1] > 8):
                    frame_no.append(df_alert["FN"].iloc[i])
                else:
                    if (df_alert["NewEMA"].iloc[i] < df_alert["NewEMA"].iloc[i-1]):
                        frame_no[-1] = df_alert["FN"].iloc[i]
    number_of_blinks.append(len(frame_no))
    # print(frame_no)
    for i in range(len(frame_no)):
        start_frame, end_frame, blinked_frame = 0, 0, 0
        blinked_frame = frame_no[i]
        earemv = df_alert["NewEMA"].loc[df_alert["FN"]== blinked_frame].iloc[0]
        # print(i)
        if i == 0:
            sf = 0
        else:
            # print("end_blink",blink_duration["End_blink"].iloc[i-1])
            sf = df_alert.index[df_alert["FN"] == blink_duration["End_blink"].iloc[i-1]][0]
        # 
        if i == len(frame_no)-1:
            en = df_alert.shape[0]
        else:
            en = df_alert.index[df_alert["FN"] == frame_no[i+1]][0]
        # print(sf)
        # print("start {} and end frame {}".format(sf,en))
        
        for j in range(df_alert.index[df_alert["FN"] == frame_no[i]][0], sf, -1):
        # print(j)
            if (df_alert["NewEMA"].iloc[j] > (earemv/0.92)) or df_alert["NewEAR"].iloc[j] > 0.40:
                start_frame = df_alert["FN"].iloc[j]
                break
            else:
                start_frame = df_alert["FN"].iloc[j]
            # print("start frame",start_frame)
                continue
        # print("threshold",(earemv/0.92))
        # print("------------")
        for j in range(df_alert.index[df_alert["FN"] == frame_no[i]][0], en):
        # print(df_alert["NewEMA"].iloc[j])
            if ((df_alert["NewEMA"].iloc[j]) > (earemv/0.92)) or (df_alert["NewEAR"].iloc[j] > 0.40):
                end_frame = df_alert["FN"].iloc[j]
                break
            else:
                end_frame = df_alert["FN"].iloc[j]
                continue
            
        # print(start_frame)
        if end_frame-start_frame > 100:
            duration = 100
        else:
            duration = end_frame-start_frame
        blink_duration.loc[blink_duration.shape[0]] = [start_frame, end_frame, blinked_frame, duration]
    
    if len(blink_duration) == 0:
        blink_avg_duration_per_minute.append(0)
    else:
        blink_avg_duration_per_minute.append(sum(blink_duration["No_of_frames"])/blink_duration.shape[0])
    no_of_blinks = blink_duration.shape[0]
    perclos_value.append(sum(blink_duration["No_of_frames"])/1800)    
    # file = blink_duration.to_csv(x.split(".")[0]+"_Blink_Duration.csv")
    frame_no.clear()
    blink_duration = blink_duration.drop(blink_duration.index[0:])

Data_day1_analysis = pd.DataFrame(columns=["File_name","MissingFrames_per_min","Blink_per_min", "blink_duration_per_min", "Perclose_per_min"])
Data_day1_analysis["File_name"] = csvfiles
Data_day1_analysis["MissingFrames_per_min"] = missing_frames_per_minutes
Data_day1_analysis["Blink_per_min"] = number_of_blinks
Data_day1_analysis["blink_duration_per_min"] = blink_avg_duration_per_minute
Data_day1_analysis["Perclose_per_min"] = perclos_value
Data_day1_analysis.to_csv("Day3_data_analysis_Enew.csv")
print(len(number_of_blinks))
print(len(blink_avg_duration_per_minute))
print(len(perclos_value))
print(len(missing_frames_per_minutes))
# file = df_alert.to_csv("Test.csv")


14
14
14
14


In [9]:
data = pd.read_csv("Day3_data_analysis_new.csv")
data.corr()

,Unnamed: 0,MissingFrames_per_min,Blink_per_min,blink_duration_per_min,Perclose_per_min,Avg_EAR
Unnamed: 0,1.000000,-0.199115,-0.020218,-0.015213,-0.037758,0.268567
MissingFrames_per_min,-0.199115,1.000000,-0.002638,0.172045,0.214989,-0.866141
Blink_per_min,-0.020218,-0.002638,1.000000,0.250012,0.781940,-0.003926
blink_duration_per_min,-0.015213,0.172045,0.250012,1.000000,0.661641,-0.113228
Perclose_per_min,-0.037758,0.214989,0.781940,0.661641,1.000000,-0.212371
Avg_EAR,0.268567,-0.866141,-0.003926,-0.113228,-0.212371,1.000000
